In [1]:
!pip install pandas scikit-learn streamlit



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 69.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 81.4 MB/s eta 0:00:00:00:0100:01


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from collections import Counter

In [2]:
dis_sym_data = pd.read_csv("/kaggle/input/doctor-specialist-recommendation-system/Original_Dataset.csv")

In [3]:
dis_sym_data.head()


,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
dis_sym_data.shape


(4920, 18)

In [5]:
columns_to_check = []
for col in dis_sym_data.columns:
    if col != 'Disease':
        columns_to_check.append(col)
symptoms = dis_sym_data.iloc[:, 1:].values.flatten()
symptoms = list(set(symptoms))

In [6]:
for symptom in symptoms:
    dis_sym_data[symptom] = dis_sym_data.iloc[:, 1:].apply(lambda row: int(symptom in row.values), axis=1)

dis_sym_data_v1 = dis_sym_data.drop(columns=columns_to_check)
dis_sym_data_v1 = dis_sym_data_v1.loc[:, dis_sym_data_v1.columns.notna()]
dis_sym_data_v1.shape

(4920, 132)

In [7]:
dis_sym_data_v1.columns = dis_sym_data_v1.columns.str.strip()
dis_sym_data_v1.columns

Index(['Disease', 'acidity', 'muscle_wasting', 'acute_liver_failure',
       'palpitations', 'distention_of_abdomen', 'stiff_neck',
       'abnormal_menstruation', 'irritability', 'coma',
       ...
       'fatigue', 'pain_behind_the_eyes', 'swollen_extremeties', 'diarrhoea',
       'hip_joint_pain', 'skin_peeling', 'nodal_skin_eruptions', 'neck_pain',
       'malaise', 'polyuria'],
      dtype='object', length=132)

In [8]:
var_mod = ['Disease']
le = LabelEncoder()
for i in var_mod:
    dis_sym_data_v1[i] = le.fit_transform(dis_sym_data_v1[i])
X = dis_sym_data_v1.drop(columns="Disease")
y = dis_sym_data_v1['Disease']

In [9]:
def class_algo(model,independent,dependent):
    model.fit(independent,dependent)
    pred = model.predict(independent)
    accuracy = metrics.accuracy_score(pred,dependent)
    print(model_name,'Accuracy : %s' % '{0:.3%}'.format(accuracy))

In [10]:
algorithms = {'Logistic Regression': 
              {"model": LogisticRegression()},
              
              'Decision Tree': 
              {"model": tree.DecisionTreeClassifier()},
              
              'Random Forest': 
              {"model": RandomForestClassifier()},
              
              'SVM':
              {"model": svm.SVC(probability=True)},
              
              'NaiveBayes' :
              {"model": GaussianNB()},
              
              'K-Nearest Neighbors' :
              {"model": KNeighborsClassifier()},
             }

In [11]:
for model_name, values in algorithms.items():
    class_algo(values["model"],X,y)

Logistic Regression Accuracy : 100.000%
Decision Tree Accuracy : 100.000%
Random Forest Accuracy : 100.000%
SVM Accuracy : 100.000%
NaiveBayes Accuracy : 100.000%
K-Nearest Neighbors Accuracy : 100.000%


In [12]:
doc_data = pd.read_csv("/kaggle/input/doctor-specialist-recommendation-system/Doctor_Versus_Disease.csv",encoding='latin1', names=['Disease','Specialist'])

In [13]:
doc_data.tail(5)


,Disease,Specialist
36,Bronchial Asthma,Pulmonologist
37,Pneumonia,Pulmonologist
38,Osteoarthristis,Rheumatologists
39,Arthritis,Rheumatologists
40,Tuberculosis,Tuberculosis


In [14]:
doc_data['Specialist'] = np.where((doc_data['Disease'] == 'Tuberculosis'),'Pulmonologist', doc_data['Specialist'])
doc_data.tail(5)

,Disease,Specialist
36,Bronchial Asthma,Pulmonologist
37,Pneumonia,Pulmonologist
38,Osteoarthristis,Rheumatologists
39,Arthritis,Rheumatologists
40,Tuberculosis,Pulmonologist


In [15]:
des_data = pd.read_csv("/kaggle/input/doctor-specialist-recommendation-system/Disease_Description.csv")
des_data.head()

,Disease,Description
0,Drug Reaction,An adverse drug reaction (ADR) is an injury ca...
1,Malaria,An infectious disease caused by protozoan para...
2,Allergy,An allergy is an immune system response to a f...
3,Hypothyroidism,"Hypothyroidism, also called underactive thyroi..."
4,Psoriasis,Psoriasis is a common skin disorder that forms...


In [16]:
test_col = []
for col in dis_sym_data_v1.columns:
    if col != 'Disease':
        test_col.append(col)


In [30]:
test_data = {}
symptoms = []
predicted = []
def test_input():
    symptoms.clear()
    predicted.clear()
    num_inputs = int(input("Enter the number of symptoms you have: "))
    for i in range(num_inputs):
        user_input = input("Enter Symptoms #{}: ".format(i+1))
        symptoms.append(user_input)
    print("Symptoms you have:", symptoms)
    for column in test_col:
        test_data[column] = 1 if column in symptoms else 0
        test_df = pd.DataFrame(test_data, index=[0])
    print("Predicting Disease based on 6 ML algorithms...")
    for model_name, values in algorithms.items():
        predict_disease = values["model"].predict(test_df)
        predict_disease = le.inverse_transform(predict_disease)
        predicted.extend(predict_disease)
    disease_counts = Counter(predicted)
    percentage_per_disease = {disease: (count / 6) * 100 for disease, count in disease_counts.items()}
    result_df = pd.DataFrame({"Disease": list(percentage_per_disease.keys()),
                               "Chances": list(percentage_per_disease.values())})
    result_df = result_df.merge(doc_data, on='Disease', how='left')
    result_df = result_df.merge(des_data, on='Disease', how='left')
    return result_df

In [31]:
pd.set_option('display.max_colwidth', None)


In [38]:
test_input() 


Enter the number of symptoms you have:  3
Enter Symptoms #1:  nausea
Enter Symptoms #2:  chills
Enter Symptoms #3:  vomiting


Symptoms you have: ['nausea', 'chills', 'vomiting']
Predicting Disease based on 6 ML algorithms...


,Disease,Chances,Specialist,Description
0,Allergy,50.000000,Allergist,"An allergy is an immune system response to a foreign substance that's not typically harmful to your body.They can include certain foods, pollen, or pet dander. Your immune system's job is to keep you healthy by fighting harmful pathogens."
1,(vertigo) Paroymsal Positional Vertigo,33.333333,Otolaryngologist,Benign paroxysmal positional vertigo (BPPV) is one of the most common causes of vertigo — the sudden sensation that you're spinning or that the inside of your head is spinning. Benign paroxysmal positional vertigo causes brief episodes of mild to intense dizziness.
2,Paralysis (brain hemorrhage),16.666667,Neurologist,"Intracerebral hemorrhage (ICH) is when blood suddenly bursts into brain tissue, causing damage to your brain. Symptoms usually appear suddenly during ICH. They include headache, weakness, confusion, and paralysis, particularly on one side of your body."


In [25]:
!pip install reportlab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.8 MB/s eta 0:00:0000:01


In [39]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from io import BytesIO
from IPython.display import FileLink
import pandas as pd
import datetime

def generate_pdf_report(symptoms, result_df, filename="Health_Report.pdf"):
    buffer = BytesIO()
    c = canvas.Canvas(buffer, pagesize=A4)
    width, height = A4

    y = height - 50
    c.setFont("Helvetica-Bold", 16)
    c.drawString(50, y, "🩺 Personalized Health Report")
    y -= 30

    # Timestamp
    c.setFont("Helvetica-Oblique", 10)
    c.drawString(50, y, f"Generated on: {datetime.datetime.now().strftime('%d-%m-%Y %H:%M:%S')}")
    y -= 30

    # Symptoms section
    c.setFont("Helvetica-Bold", 12)
    c.drawString(50, y, "Symptoms:")
    y -= 20
    c.setFont("Helvetica", 11)
    for i, symptom in enumerate(symptoms, 1):
        c.drawString(70, y, f"{i}. {symptom}")
        y -= 18

    y -= 20
    c.setFont("Helvetica-Bold", 12)
    c.drawString(50, y, "Prediction Results:")
    y -= 20

    for idx, row in result_df.iterrows():
        c.setFont("Helvetica-Bold", 11)
        c.drawString(50, y, f"{idx + 1}. Disease: {row['Disease']} ({row['Chances']:.2f}%)")
        y -= 18
        c.setFont("Helvetica", 10)
        c.drawString(70, y, f"Specialist: {row['Specialist']}")
        y -= 15
        desc_lines = row['Description'][:180].split('. ')
        for line in desc_lines:
            c.drawString(70, y, f"- {line.strip()}")
            y -= 14
        y -= 10

        if y < 100:
            c.showPage()
            y = height - 50

    c.save()
    buffer.seek(0)

    # Save to disk in Kaggle environment
    with open(filename, "wb") as f:
        f.write(buffer.read())

    print("✅ PDF report generated.")
    return filename


In [40]:
report_file = generate_pdf_report(symptoms, result_df)
FileLink(report_file)


✅ PDF report generated.


/kaggle/working/Health_Report.pdf